# Mend Applications

https://api-docs.mend.io/platform/3.0/applications/getorganizationapplications

In [ ]:
ALTER SESSION SET TIMEZONE = 'Europe/London';

In [ ]:
import os
import requests
import pandas as pd
from json import loads, dumps

try:
    
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    results = session.sql("SELECT PROD.RAW.mend_orgUuid_jwtToken()").collect()
    orgUuid = results[0][0]

    
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    results = session.sql("SELECT PROD.RAW.mend_user_key_jwtToken()").collect()
    userKey = results[0][0]

    
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    results = session.sql("SELECT PROD.RAW.mend_email_jwtToken()").collect()
    email = results[0][0]
    
except:
    from dotenv import load_dotenv
    load_dotenv()
    orgUuid = os.getenv('orgUuid')
    userKey = os.getenv('userKey')
    email = os.getenv('email')


In [ ]:
class Mend:
    
    def __init__(self):
        self.orgUuid = orgUuid
        self.userKey = userKey
        self.email = email
        self.login_body = {
            "email": email,
            "orgUuid": orgUuid,
            "userKey": userKey
        }
        self.url_applications = f"https://api-saas-eu.whitesourcesoftware.com/api/v3.0/orgs/{self.orgUuid}"

    ###################################################
    # AUTH
    ###################################################
    def authenticate_user(self):
        """
        Authenticates user and returns JWT token
        """
        # Endpoint do logowania
        url_login = "https://api-saas-eu.whitesourcesoftware.com/api/v3.0/login"
        
        # Wykonanie żądania POST do logowania
        response_login = requests.post(url_login, json=self.login_body)
        response_login.raise_for_status()
        login_data = response_login.json()
        refresh_token = login_data['response']['refreshToken']
        
        # Uzyskanie access token'a używającego refresh token'a
        url_access_token = f"{url_login}/accessToken"
        refresh_token_headers = {
            "wss-refresh-token": refresh_token
        }
        
        # Wykonanie żądania POST do uzyskania JWT tokena
        access_token_response = requests.post(url_access_token, headers=refresh_token_headers)
        access_token_response.raise_for_status()
        access_token_data = access_token_response.json()
        
        # Fixed: You need to check the actual response structure
        # This might be 'jwtToken', 'accessToken', or another key
        jwt_token = access_token_data['response']['jwtToken']  # Update this based on actual API response
        
        return jwt_token

    def get_headers(self):
        headers = {
            "Authorization": f"Bearer {self.authenticate_user()}"
        }
        return headers

    ###################################################
    # BASIC
    ###################################################
    def get_applications(self):
        headers_applications = {
            "Authorization": f"Bearer {self.authenticate_user()}"
        }
        
        url_applications = f"{self.url_applications}/applications"
        all_aplications = []
        applications_cursor = None
        
        # Initial parameters - max limit is 10000 according to documentation
        applications_params = {
            "limit": 1000,  # Requesting 1000 items per page
            "cursor": None,  # If Cursor is not empty it indicates there is more data to fetch
            # and we need to iterate through all pages
        }
        
        while True:
            # Add the cursor to the parameters if it exists
            if applications_cursor:
                applications_params["cursor"] = applications_cursor
        
            # making request
            response_applications = requests.get(url_applications, headers=self.get_headers(), params=applications_params)
            applications_data = response_applications.json()
        
            # extracting data and extending all_aplications list
            applications = applications_data.get('response', [])
            all_aplications.extend(applications)
        
            # Check if there is a next page
            
            # if there is another page output would look like this:
            # {'additionalData': {'totalItems': 96,
            # 'paging': {'next': 'https://api-saas-eu.whitesourcesoftware.com/api/v3.0/orgs/71003aaf-b8e7-4e7b-a622-736b775a0eff/applications?limit=50&cursor=1'}},
            
            # if there is no additional page:
            # {'additionalData': {'totalItems': 96, 'paging': {}},
            cursor = applications_data['additionalData'].get('paging', {}).get('next')
            if not cursor:
                break
        
        # ---- # ---- Endpoint specific logic ---- # ----
        
        if all_aplications:  # Check if data was actually fetched
            all_applications_df = pd.DataFrame(all_aplications)
        else:
            print("\nNo application data fetched, temporary table not created.")
            all_applications_df = pd.DataFrame()
            
        return all_applications_df 

    def get_projects(self):
        headers = {
            'Authorization': f'Bearer {self.authenticate_user()}',
        }
    
        base_projects_url = f"{self.url_applications}/projects"
        all_projects = []  # empty list to store all projects
        has_more = True  # flag to control pagination
        cursor = None  # cursor for pagination
    
        # Initial parameters - max limit is 10000 according to documentation
        params = {
            "limit": "1000",  # Requesting 1000 items per page
            "populateApplications": "true"  # Include application details in the response
        }
    
        print("Fetching projects data with pagination...")
    
        # Continue fetching while there are more pages
        while has_more:
            # Add cursor to parameters if we're not on the first page
            if cursor:
                params["cursor"] = cursor
            
            # Make the request
            response_projects = requests.get(base_projects_url, headers=self.get_headers(), params=params)
            
            # Check if the request was successful
            if response_projects.status_code != 200:
                # print(f"Error fetching projects data: {response_projects.status_code}")
                # print(response_projects.text)
                break
            
            # Parse the JSON response
            response_data = response_projects.json()
            
            # Extract projects data from the response
            projects = response_data.get('response', [])
            
            # If we got no projects or empty response, break the loop
            if not projects:
                has_more = False
                continue
            
            # Append the current page's data to the list
            all_projects.extend(projects)
            # print(f"Retrieved {len(projects)} projects. Total so far: {len(all_projects)}")
            
            # Check if there's a cursor for the next page
            additional_data = response_data.get('additionalData', {})
            cursor = additional_data.get('cursor')
            
            # If no cursor is returned, we've reached the end
            if not cursor:
                has_more = False
    
        projects_data = pd.DataFrame(all_projects)
        
        print(projects_data.reset_index().columns)
    
        return projects_data

    def get_alert_types(self):
    
        url_alert_types = f"https://api-saas-eu.whitesourcesoftware.com/api/v2.0/orgs/{self.orgUuid}/summary/alertTypes"
        response_alert_types = requests.get(url_alert_types, headers=self.get_headers())
        alert_types_data = response_alert_types.json()
    
        # ---- # ---- Endpoint specific logic ---- # ----
    
        # Create Pandas DataFrame
        df = pd.DataFrame(alert_types_data).reset_index().rename(columns={'index': 'PATH'})
        df = df[['PATH', 'retVal']]
        df['row'] = 0
        wide1 = df.pivot(index='row', columns='PATH', values='retVal').reset_index().drop('row', axis=1)
    
        return wide1

    def get_vulnerabilities_project(self):
        vulnerabilities_project = []
        page = 0
        page_size = 10000
        is_last_page = False
        headers = {
            'Authorization': f'Bearer {self.authenticate_user()}',
        }
        
        while not is_last_page:
            url = f"https://api-saas-eu.whitesourcesoftware.com/api/v2.0/orgs/{orgUuid}/summary/projects/vulnerableLibraryCount?search=projectName:like:"
                    
            params = {
                "page": str(page),
                "pageSize": str(page_size)
            }
                    
            response = requests.get(url, headers=self.get_headers(), params=params)
            data = response.json()
                    
            print(f"Page {page} Vulnerabilities Project: ", data)
            vulnerabilities_project.append(data)
            page += 1
            is_last_page = data['additionalData']['isLastPage']
        
        vulnerabilities_project_df = pd.DataFrame(vulnerabilities_project)['retVal'].iloc[0]
        vulnerabilities_project_df = pd.DataFrame(vulnerabilities_project_df)
        
        return vulnerabilities_project_df
    
    def get_alerts_severity(self):
        all_aplications = []
        applications_cursor = None

        url_alerts_severity = f"https://api-saas-eu.whitesourcesoftware.com/api/v2.0/orgs/{orgUuid}/summary/alertCountPerSeverity"
        
        response_alerts_severity = requests.get(url_alerts_severity, headers=self.get_headers())
        alerts_severity_data = response_alerts_severity.json()
        print("Alerts Severity: ", alerts_severity_data)
        df_alerts_severity_data = pd.DataFrame([alerts_severity_data['retVal']])
       
        return df_alerts_severity_data

    def get_labels(self):
        # ——— ## ———— ## ————
        # Stream: Labels
        # This stream gets labels associated with the organization.
        url_labels = f"{self.url_applications}/labels"
        response_labels = requests.get(url_labels, headers=self.get_headers())  # Fixed: removed extra parenthesis
        labels_data = response_labels.json()
    
        labels_data = pd.DataFrame(labels_data['response'])
    
        # Assume df_labels holds the final DataFrame for this notebook
        # <<<< ADJUST THE DATAFRAME VARIABLE NAME BELOW IF IT'S DIFFERENT >>>>
    
        # Display the DataFrame (optional, for confirmation in notebooks)
        return labels_data
    
    ###################################################
    # UNIQUES
    ###################################################
    def get_unique_product_uuids(self):
        unique_product_uuids = self.get_applications()['uuid'].unique().tolist()
        return unique_product_uuids
        
    def get_unique_uuid(self):
        unique_uuid = self.get_projects()['uuid'].unique().tolist()
        return unique_uuid  # Fixed: was returning undefined 'unique_projects'
        
    def get_unique_policy_violations(self):
        policy_violations = self.get_projects()['applicationUuid'].tolist()
        return policy_violations
        
    ###################################################
    # LOOPS
    ###################################################
    def get_alerts_per_library(self):
        all_security_alerts_data = []
        page_size = 1000  # Using a smaller page size for example, adjust as needed (max 10000)
        
        for project_token in self.get_unique_uuid():
            page = 0
            while True:
                url_security_alerts_library = f"https://api-saas-eu.whitesourcesoftware.com/api/v2.0/projects/{project_token}/alerts/security/groupBy/component"
                params = {"pageSize": str(page_size), "page": str(page)}
                response = requests.get(url_security_alerts_library, headers=self.get_headers(), params=params)
                # Raise an exception for bad status codes (4xx client error or 5xx server error)
                response.raise_for_status()
                data_page = response.json()
        
                # Extract the list of alerts, default to empty list if 'retVal' is missing or null
                alerts = data_page.get('retVal', [])
        
                # Add project token to each alert record for context
                for alert in alerts:
                    alert['projectToken'] = project_token
                all_security_alerts_data.extend(alerts)
        
                # If the number of returned alerts is less than the page size, it's the last page
                if len(alerts) < page_size:
                    break
        
                # Increment page number for the next request
                page += 1
        
        df_security_alerts = pd.DataFrame(all_security_alerts_data)
        return df_security_alerts
    
    def get_alerts_per_project(self):
        all_security_findings = []  # Initialize list to store all findings from all projects

        for project_uuid in self.get_unique_uuid():
            url_security_alerts_project = f"https://api-saas-eu.whitesourcesoftware.com/api/v3.0/projects/{project_uuid}/dependencies/findings/security"
            params = {"limit": "10000"}
            response_security_alerts_project = requests.get(url_security_alerts_project, headers=self.get_headers(), params=params)
            security_alerts_project_data = response_security_alerts_project.json()['response']
            all_security_findings.extend(security_alerts_project_data)
        
        df_final_alerts = pd.DataFrame(all_security_findings)
        # Now df_final_alerts contains all findings from all projects
        # You can display it or process it further
        df_final_alerts['status'] = df_final_alerts['findingInfo'].apply(lambda x: x.get('status') if x else None)
        df_final_alerts['detected_at'] = df_final_alerts['findingInfo'].apply(lambda x: x.get('detectedAt') if x else None)
        df_final_alerts['modified_at'] = df_final_alerts['findingInfo'].apply(lambda x: x.get('modifiedAt') if x else None)
        df_final_alerts.drop(columns=['findingInfo'])
        
        list_of_alerts_cleaned = df_final_alerts[['name', 'type', 'uuid', 'topFix', 'project', 'component',
                                                   'application', 'exploitable', 'vulnerability', 'threatAssessment', 
                                                   'scoreMetadataVector', 'status', 'detected_at', 'modified_at']]
        
        # Display the DataFrame (optional, for confirmation in notebooks)
        # list_of_alerts_cleaned.columns
        return list_of_alerts_cleaned
        
    def get_libraries(self):
        params = {
            "limit": "10000",  # Requesting 1000 items per page
        }
    
        all_libraries = []  # Empty list to store all libraries
        # count_libraries = 0
    
        # Iterate over each project UUID
        for project_uuid in self.get_unique_uuid():
            # print(f"Fetching library no {count_libraries}")
            # count_libraries += 1
            # print(f"Fetching libraries for project {project_uuid}...")
            has_more = True  # Reset pagination flag for each project
            cursor = None  # Reset cursor for each project
            
            while has_more:
                # Add cursor to parameters if we're not on the first page
                if cursor:
                    params["cursor"] = cursor
                
                # Construct the URL for fetching libraries
                url_libraries = f"https://api-saas-eu.whitesourcesoftware.com/api/v3.0/projects/{project_uuid}/dependencies/libraries"
                response_libraries = requests.get(url_libraries, headers=self.get_headers(), params=params)
                
                # Check if the request was successful
                if response_libraries.status_code != 200:
                    # print(f"Error fetching libraries for project {project_uuid}: {response_libraries.status_code}")
                    print(response_libraries.text)
                    break
                
                # Parse the JSON response
                libraries_data = response_libraries.json()
                
                # Extract libraries from the response
                libraries = libraries_data.get('response', [])
                
                # Add project_uuid to each library
                for library in libraries:
                    library['project_uuid'] = project_uuid
                
                # Append the current page's data to the list
                all_libraries.extend(libraries)
                # print(f"Retrieved {len(libraries)} libraries for project {project_uuid}. Total so far: {len(all_libraries)}")
                
                # Check if there's a cursor for the next page
                additional_data = libraries_data.get('additionalData', {})
                cursor = additional_data.get('cursor')
                
                # If no cursor is returned, we've reached the end
                has_more = bool(cursor)
                
        all_libraries_df = pd.DataFrame(all_libraries)
        # wide1 = wide1.drop('row', axis = 1)
    
        return all_libraries_df
    
    def get_license_policy_violations(self):
        policy_violations_list = []
        # for product_uuid in unique_product_uuids:
        for product_uuid in self.get_unique_policy_violations():  # lub unique_product_uuids
            page = 0
            is_last_page = False
        
            while not is_last_page:
                params = {
                    'pageSize': '10000',
                    'page': str(page)
                }
                url_licence_policy_violations = (
                    f"https://api-saas-eu.whitesourcesoftware.com/api/v2.0/products/{product_uuid}/alerts/legal"
                )
                response = requests.get(url_licence_policy_violations, headers=self.get_headers(), params=params)
                data = response.json()
        
                # Dodaj wyniki z tej strony do listy
                if 'retVal' in data and isinstance(data['retVal'], list):
                    policy_violations_list.extend(data['retVal'])
        
                # Sprawdź czy to ostatnia strona
                is_last_page = data.get('body', {}).get('additionalData', {}).get('isLastPage', True)
                # Jeśli nie ma body, spróbuj bezpośrednio z głównego poziomu (dostosuj w zależności od API)
                if not isinstance(is_last_page, bool):
                    is_last_page = data.get('additionalData', {}).get('isLastPage', True)
        
                page += 1
        
        # Tworzenie DataFrame z zebranych danych
        policy_violations_df = pd.DataFrame(policy_violations_list)
        
        # Jeżeli alertInfo jest słownikiem, wyciągnij potrzebne pola
        policy_violations_df['status'] = policy_violations_df['alertInfo'].apply(lambda x: x.get('status') if isinstance(x, dict) else None)
        policy_violations_df['detected_at'] = policy_violations_df['alertInfo'].apply(lambda x: x.get('detectedAt') if isinstance(x, dict) else None)
        policy_violations_df['modified_at'] = policy_violations_df['alertInfo'].apply(lambda x: x.get('modifiedAt') if isinstance(x, dict) else None)
        
        policy_violations_df = policy_violations_df[[
            'name', 'type', 'uuid', 'project', 'component',
            'policyName', 'status', 'detected_at', 'modified_at'
        ]]
        
        # Display the DataFrame (optional, for confirmation in notebooks)
        policy_violations_df = policy_violations_df.drop_duplicates(subset=['uuid'])
        policy_violations_df = policy_violations_df.reset_index(drop=True)
        return policy_violations_df
        
    def get_project_due_diligence(self):
        all_due_diligence = []  # empty list to store all due diligence data
        params = {
            'limit': "10000"
        }
        for i in self.get_unique_uuid():
            due_diligence_url = f"https://api-saas-eu.whitesourcesoftware.com/api/v3.0/projects/{i}/dependencies/libraries/licenses"
            response_due_diligence = requests.get(due_diligence_url, headers=self.get_headers(), params=params) 
            due_diligence = response_due_diligence.json().get('response', [])
            all_due_diligence.extend(due_diligence)
        
        return pd.DataFrame(all_due_diligence)


###################################################
# CLASS INSTANTIATION
###################################################
# To use the class:
api = Mend()

In [ ]:
auth = api.authenticate_user()
auth

In [ ]:
applications = api.get_applications()
applications


In [ ]:
applications_table = 'PYTHON_APPLICATIONS'

session.write_pandas(applications,
                         table_name=applications_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
-- CREATE OR REPLACE TABLE prod.raw.mend_applications
-- (uuid varchar,
-- name varchar,
-- timestamp timestamp_tz);

-- truncate prod.raw.mend_applications;

insert overwrite into prod.raw.mend_applications
SELECT
    "uuid", -- Use quotes if column names are case-sensitive
    "name",
    current_timestamp as timestamp
FROM PYTHON_APPLICATIONS;

# Mend Projects

https://api-docs.mend.io/platform/3.0/projects/getorganizationprojects

In [ ]:
projects = api.get_projects()
projects

In [ ]:
projects_table = 'PYTHON_PROJECTS'

session.write_pandas(projects,
                         table_name=projects_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
-- create or replace table prod.raw.mend_projects 
-- (
-- name varchar,
-- path varchar,
-- uuid varchar,
-- applicationname varchar,
-- applicationuuid varchar,
-- timestamp timestamp_tz);

-- -- select 
-- -- value:name::string as name,
-- -- value:path::string as path,
-- -- value:uuid::string as uuid,
-- -- value:applicationName::string as applicationName,
-- -- value:applicationUuid::string as applicationUuid
-- -- from table (flatten(input => get_mend_projects()));


-- truncate prod.raw.mend_projects;

insert overwrite into prod.raw.mend_projects
SELECT
    "name", -- Use quotes if column names are case-sensitive
    "path",
    "uuid",
    "applicationName",
    "applicationUuid",
    current_timestamp as timestamp
FROM PYTHON_PROJECTS;

-- SELECT top 15 * from prod.raw.mend_projects 

# Mend Alert Types

https://api-docs.mend.io/sca/2.0/summary-organization/getorganizationalerttypesummaries

In [ ]:
alert_types = api.get_alert_types()

In [ ]:
alert_types_table = 'PYTHON_ALERT_TYPES'

session.write_pandas(alert_types,
                         table_name=alert_types_table,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
-- create or replace table prod.raw.mend_alert_types
-- (
-- libraries variant,
-- policies variant,
-- securityPerLibrary variant,
-- securityPerVulnerability variant,
-- timestamp timestamp_tz
-- );

-- truncate prod.raw.mend_alert_types;

insert overwrite into prod.raw.mend_alert_types
select 
"libraries",
"policies",
"securityPerLibrary",
"securityPerVulnerability",
current_timestamp as timestamp
FROM PYTHON_ALERT_TYPES;

-- SELECT top 15 * from prod.raw.mend_alert_types 

# Mend Alerts Severity

https://api-docs.mend.io/sca/2.0/summary-organization/getorganizationalertcountperseverity

In [ ]:
alerts_severity = api.get_alerts_severity()

In [ ]:

ALERTS_SEVERITY_TABLE = 'PYTHON_ALERTS_SEVERITY'

session.write_pandas(alerts_severity,
                            table_name=ALERTS_SEVERITY_TABLE,
                            auto_create_table=True,
                            overwrite=True,
                            table_type="temporary")

In [ ]:
-- create or replace table prod.raw.mend_alerts_severity(
-- low string,
-- high string,
-- total string,
-- medium string,
-- critical string,
-- timestamp timestamp_tz
-- );

-- truncate prod.raw.mend_alerts_severity;

insert overwrite into prod.raw.mend_alerts_severity
select 
"low",
"high",
"total",
"medium",
"critical",
current_timestamp as timestamp
from PYTHON_ALERTS_SEVERITY;

-- select top 15 * from prod.raw.mend_alerts_severity

# Mend libraries

https://api-docs.mend.io/test/mend_api_3.0_swagger/findings-project/getlibraries

In [ ]:
libraries = api.get_libraries()


In [ ]:

LIBRARIES_TABLE = 'PYTHON_LIBRARIES'

session.write_pandas(libraries,
                            table_name=LIBRARIES_TABLE,
                            auto_create_table=True,
                            overwrite=True,
                            table_type="temporary")

In [ ]:
-- create or replace table prod.raw.mend_libraries 
-- (name  string,
-- purl  string,
-- sha1  string,
-- type  string,
-- uuid  string,
-- groupid  string,
-- version  string,
-- licenses  variant,
-- extension  string,
-- locations  string,
-- artifactId  string,
-- classifier  string,
-- project_uuid  string,
-- description  string,
-- libraryType  string,
-- architecture  string,
-- languageVersion  string,
-- directDependency  boolean,
-- copyrightReferences variant,
-- timestamp timestamp_tz);

-- truncate table prod.raw.mend_libraries;

insert overwrite into prod.raw.mend_libraries 
select 
"name",
"purl",
"sha1",
"type",
"uuid",
"groupId",
"version" ,
"licenses",
"extension",
"locations",
"artifactId",
"classifier",
"project_uuid",
"description",
"libraryType",
"architecture",
"languageVersion",
"directDependency",
"copyrightReferences",
current_timestamp as timestamp
from PYTHON_LIBRARIES;

-- select top 15 * from prod.raw.mend_libraries

# Mend labels

https://api-docs.mend.io/platform/3.0/administration-labels/getlabels

In [ ]:
labels = api.get_labels()
labels

In [ ]:

LABELS_TABLE = 'PYTHON_LABELS'

session.write_pandas(labels,
                            table_name=LABELS_TABLE,
                            auto_create_table=True,
                            overwrite=True,
                            table_type="temporary")

In [ ]:
-- create or replace table prod.raw.mend_labels
-- (id int,
-- uuid string,
-- value string, 
-- system boolean,
-- createdAt timestamp_tz,
-- createdBy string, 
-- namespace string, 
-- applications int,
-- projects int,
-- timestamp timestamp_tz);

-- truncate table prod.raw.mend_labels;

insert overwrite into prod.raw.mend_labels
select
"id",
"uuid",
"value" , 
"system" ,
"createdAt" ,
"createdBy" , 
"namespace" , 
"applications" ,
"projects" ,
current_timestamp as timestamp
from 
PYTHON_LABELS;

-- select top 15 * from prod.raw.mend_labels

# Mend Security alerts per project

https://api-docs.mend.io/platform/3.0/findings-project/getsecurityvulnerabilityfindings

In [ ]:
# 

alerts_per_project = api.get_alerts_per_project()
alerts_per_project

In [ ]:

ALERTS_PER_PROJECT_TABLE = 'PYTHON_ALERTS_PER_PROJECT'

session.write_pandas(alerts_per_project,
                            table_name=ALERTS_PER_PROJECT_TABLE,
                            auto_create_table=True,
                            overwrite=True,
                            table_type="temporary")

In [ ]:
-- create or replace table prod.raw.mend_security_alerts_per_project
-- (
-- name string,
-- uuid string,
-- topFix object,
-- project object,
-- component object,
-- -- effective string,
-- application object,
-- exploitable boolean,
-- vulnerability object,
-- threatAssessment object,
-- scoreMetadataVector string,
-- status string,
-- detected_at  timestamp_tz,
-- modified_at timestamp_tz,
-- type string,
-- timestamp timestamp_tz
-- );

-- truncate table prod.raw.mend_security_alerts_per_project;


insert overwrite into prod.raw.mend_security_alerts_per_project
select 
"name" ,
"uuid" ,
"topFix" ,
"project" ,
"component" ,
-- "effective" ,
"application" ,
"exploitable" ,
"vulnerability" ,
"threatAssessment" ,
"scoreMetadataVector" ,
"status" ,
"detected_at"  ,
"modified_at" ,
"type" ,
current_timestamp as timestamp
from PYTHON_ALERTS_PER_PROJECT;

-- select top 15 * from prod.raw.mend_security_alerts_per_project


# Mend Security alerts per Library

https://api-docs.mend.io/sca/2.0/alerts-project/getlibrarysecurityvulnerabilityalerts

In [ ]:

# unique_uuid = projects['uuid'].unique().tolist()

alerts_per_library = api.get_alerts_per_library()
alerts_per_library

In [ ]:

PYTHON_ALERTS_PER_LIBRARY_TABLE = 'PYTHON_ALERTS_PER_LIBRARY'

session.write_pandas(alerts_per_library,
                     table_name=PYTHON_ALERTS_PER_LIBRARY_TABLE,
                     auto_create_table=True,
                     overwrite=True,
                     table_type="temporary")


In [ ]:

-- create or replace table prod.raw.mend_security_alerts_per_library
-- (
-- name string,
-- uuid string,
-- total int,
-- lowNum int,
-- status string,
-- highNum int,
-- product variant,
-- project variant,
-- lastScan timestamp_tz,
-- severity string,
-- component variant,
-- effective string,
-- ignoreNum int,
-- malicious boolean,
-- mediumNum int,
-- detectedAt timestamp_tz,
-- modifiedAt timestamp_tz,
-- criticalNum int,
-- exploitable boolean,
-- libraryType string,
-- publishedAt timestamp_tz,
-- reachability string,
-- numOfMalicious int,
-- numOfReachable int,
-- lastCveUpdatedAt timestamp_tz,
-- exploitCodeMaturity string,
-- timestamp timestamp_tz
-- );

-- truncate table prod.raw.mend_security_alerts_per_library;

insert overwrite into prod.raw.mend_security_alerts_per_library
select
"name",
"uuid",
"total",
"lowNum",
"status",
"highNum",
"product",
"project",
"lastScan",
"severity",
"component",
"effective",
"ignoreNum",
"malicious",
"mediumNum",
"detectedAt",
"modifiedAt",
"criticalNum",
"exploitable",
"libraryType",
"publishedAt",
"reachability",
"numOfMalicious",
"numOfReachable",
"lastCveUpdatedAt",
"exploitCodeMaturity",
current_timestamp as timestamp
from PYTHON_ALERTS_PER_LIBRARY;

-- select top 15 * from prod.raw.mend_security_alerts_per_library

# Mend Vulnerabilities Project
https://api-docs.mend.io/sca/2.0/summary-organization/getvulnerablelibscountbyprojects

In [ ]:
vulnerabilities_project = api.get_vulnerabilities_project()


In [ ]:
VULNERABILITIES_PROJECT_TABLE = 'PYTHON_VULNERABILITIES_PROJECT'

session.write_pandas(vulnerabilities_project,
                     table_name=VULNERABILITIES_PROJECT_TABLE,
                     auto_create_table=True,
                     overwrite=True,
                     table_type="temporary")

In [ ]:
-- create or replace table prod.raw.mend_vulnerabilities_project
-- (
-- uuid string,
-- productName string,
-- projectName string,
-- vulnerableLibraries int,
-- timestamp timestamp_tz
-- );

-- truncate table prod.raw.mend_vulnerabilities_project;

insert overwrite into prod.raw.mend_vulnerabilities_project
select
"uuid",
"productName",
"projectName",
"vulnerableLibraries",
current_timestamp as timestamp
from PYTHON_VULNERABILITIES_PROJECT;

-- select top 15 * from prod.raw.mend_vulnerabilities_project

# Mend Project Due Diligence
https://api-docs.mend.io/test/mend_api_3.0_swagger/findings-project/getduediligenceinfobymultiplecontexts

In [ ]:
# unique_uuid = projects['uuid'].unique().tolist()

project_due_diligence = api.get_project_due_diligence()
project_due_diligence

In [ ]:

PROJECT_DUE_DILIGENCE_TABLE = 'PYTHON_PROJECT_DUE_DILIGENCE'

session.write_pandas(project_due_diligence,
                         table_name=PROJECT_DUE_DILIGENCE_TABLE,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
-- create or replace table prod.raw.mend_project_due_diligence
-- (
-- uuid string,
-- name string,
-- license variant,
-- project variant,
-- component variant,
-- extradata variant,
-- riskscore variant,
-- copyrights string,
-- timestamp timestamp_tz
-- );

-- truncate table prod.raw.mend_project_due_diligence;


insert overwrite into prod.raw.mend_project_due_diligence
select
"uuid",
"name",
"license",
"project",
"component",
"extraData",
"riskScore",
"copyrights",
current_timestamp as timestamp
from PYTHON_PROJECT_DUE_DILIGENCE
where "uuid" is not null;

-- select top 15 * from prod.raw.mend_project_due_diligence


# Mend License Policy Violations
https://api-docs.mend.io/sca/2.0/alerts-project/getlegalandcompliancealerts

In [ ]:
# policy_violations = projects['applicationUuid'].tolist()

license_policy_violations = api.get_license_policy_violations()
license_policy_violations
                                                                   

In [ ]:
LICENSE_POLICY_VIOLATIONS_TABLE = 'PYTHON_LICENCE_POLICY_VIOLATIONS'

session.write_pandas(license_policy_violations,
                     table_name=LICENSE_POLICY_VIOLATIONS_TABLE,
                     auto_create_table=True,
                     overwrite=True,
                     table_type="temporary")


In [ ]:
-- create or replace table prod.raw.mend_license_policy_violations
-- (
-- name string,
-- type string,
-- uuid string,
-- project variant,
-- component variant,
-- policyName string,
-- status string,
-- detected_at timestamp_tz,
-- modified_at timestamp_tz,
-- timestamp timestamp_tz
-- );

-- truncate table prod.raw.mend_license_policy_violations;


insert overwrite into prod.raw.mend_license_policy_violations
select
"name",
"type",
"uuid",
"project",
"component",
"policyName",
"status",
"detected_at",
"modified_at",
current_timestamp as timestamp
from PYTHON_LICENCE_POLICY_VIOLATIONS;

-- select top 15 * from prod.raw.mend_license_policy_violations